# Getting data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import warnings
from ML_utils import *
warnings.filterwarnings("ignore")

In [3]:
df=pd.read_excel('processminer-rare-event-mts.xlsx')

In [4]:
df.head()

,time,y,x1,x2,x3,x4,x5,x6,x7,x8,...,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61
0,1999-05-01 00:00:00,0,0.376665,-4.596435,-4.095756,13.497687,-0.118830,-20.669883,0.000732,-0.061114,...,10.091721,0.053279,-4.936434,-24.590146,18.515436,3.473400,0.033444,0.953219,0.006076,0
1,1999-05-01 00:02:00,0,0.475720,-4.542502,-4.018359,16.230659,-0.128733,-18.758079,0.000732,-0.061114,...,10.095871,0.062801,-4.937179,-32.413266,22.760065,2.682933,0.033536,1.090502,0.006083,0
2,1999-05-01 00:04:00,0,0.363848,-4.681394,-4.353147,14.127997,-0.138636,-17.836632,0.010803,-0.061114,...,10.100265,0.072322,-4.937924,-34.183774,27.004663,3.537487,0.033629,1.840540,0.006090,0
3,1999-05-01 00:06:00,0,0.301590,-4.758934,-4.023612,13.161566,-0.148142,-18.517601,0.002075,-0.061114,...,10.104660,0.081600,-4.938669,-35.954281,21.672449,3.986095,0.033721,2.554880,0.006097,0
4,1999-05-01 00:08:00,0,0.265578,-4.749928,-4.333150,15.267340,-0.155314,-17.505913,0.000732,-0.061114,...,10.109054,0.091121,-4.939414,-37.724789,21.907251,3.601573,0.033777,1.410494,0.006105,0


In [5]:
df['y'].value_counts()

0    18274
1      124
Name: y, dtype: int64

In [6]:
df.isna().any().value_counts()

False    63
dtype: int64

In [7]:
#Defining features and targets for Performance Evaluator class
X=df.drop(['time','y'],axis=1).values
y=df['y'].values


In [14]:
#Pipeline com normalizacao e classificador 
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import tree

pip = Pipeline([('std',StandardScaler()),('clf',tree.DecisionTreeClassifier())])

In [15]:
cv=Cross_valid_clf(X,y,n_splits=5,stratified=False)

In [16]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()

In [17]:
cv.score(lr,verbose=True)

------------------------------

Fold 1 out of 5
Accuracy_score: 0.8317934782608696
------------------------------

Fold 2 out of 5
Accuracy_score: 0.997554347826087
------------------------------

Fold 3 out of 5
Accuracy_score: 0.9883152173913043
------------------------------

Fold 4 out of 5
Accuracy_score: 0.9953791791247621
------------------------------

Fold 5 out of 5
Accuracy_score: 0.9923892362054906

 Overall Score:
Accuracy_score Mean: 0.9610862917617027   Std: 0.06472069465258391


0.9610862917617027

In [21]:
cv.f1score(lr,verbose=True)

------------------------------

Fold 1 out of 5
f1_score: 0.06354009077155824
------------------------------

Fold 2 out of 5
f1_score: 0.39999999999999997
------------------------------

Fold 3 out of 5
f1_score: 0.46913580246913583
------------------------------

Fold 4 out of 5
f1_score: 0.3703703703703703
------------------------------

Fold 5 out of 5
f1_score: 0.33333333333333337

 Overall f1score:
f1score Mean: 0.32727591938887957   Std: 0.13918579639735684


0.32727591938887957

In [22]:
cv.fbetascore(pip,verbose=True)

------------------------------

Fold 1 out of 5
fbeta_score: 0.33847685415629664
------------------------------

Fold 2 out of 5
fbeta_score: 0.04669046965119472
------------------------------

Fold 3 out of 5
fbeta_score: 0.6094339622641509
------------------------------

Fold 4 out of 5
fbeta_score: 0.3958090803259604
------------------------------

Fold 5 out of 5
fbeta_score: 0.16630434782608694

 Overall Score:
fbeta_score Mean: 0.3113429428447379   Std: 0.19380720264995754


0.3113429428447379

In [21]:
X_train,X_test,y_train,y_test=train_test_split(X,y, random_state=32)

In [22]:
pip.fit(X_train,y_train) # treinando o modelo

Pipeline(memory=None,
     steps=[('std', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'))])

In [23]:
y_pred=pip.predict(X_test) # prediçoes do modelo

In [24]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4564
           1       0.64      0.58      0.61        36

   micro avg       0.99      0.99      0.99      4600
   macro avg       0.82      0.79      0.80      4600
weighted avg       0.99      0.99      0.99      4600



In [13]:
from sklearn.linear_model import LogisticRegression 

In [25]:
lr=LogisticRegression()
pe.score(lr)

0.06354009077155824
0.4635400907715582
0.932675893240694
1.3030462636110642
1.6363795969443977


0.32727591938887957